In [109]:
from pyspark.sql import SparkSession
from pyspark import SparkContext

SpSession = SparkSession \
    .builder \
    .appName("Pyspark Canary Turists") \
    .getOrCreate()

SpSession.sparkContext.setLogLevel("ERROR")

In [110]:
from pyspark.sql.functions import *

## Carga de datos

In [157]:
import pandas as pd
df_pd_known = pd.read_csv("https://datos.canarias.es/catalogos/estadisticas/dataset/f6c8c3ad-0426-4c7e-8fb5-93405e5ee981/resource/de9efe88-6f18-4e87-bebb-1621e654604a/download/c00028a_dsc_0113.csv")
df_sp_known = SpSession.createDataFrame(df_pd_known)
df_pd_money = pd.read_csv("https://datos.canarias.es/catalogos/estadisticas/dataset/db07c0a2-4e49-4567-902f-967ce597c077/resource/4bc718af-77a6-468d-8442-afabc1c5fb09/download/c00028a_dsc_0046.csv")
df_sp_money = SpSession.createDataFrame(df_pd_money)
df_pd_stay = pd.read_csv("https://datos.canarias.es/catalogos/estadisticas/dataset/04f2a069-17f0-44ea-8b61-3d998d7c305d/resource/82622785-6a8f-4db9-a520-794526ce957e/download/c00028a_dsc_0138.csv")
df_sp_stay = SpSession.createDataFrame(df_pd_stay)
df_pd_canal = pd.read_csv("https://datos.canarias.es/catalogos/estadisticas/dataset/eac981c4-ba6e-429f-a94d-2e3b6237b032/resource/4590bec9-9f7a-4344-b92d-da3c988cebde/download/c00028a_dsc_0119.csv")
df_sp_canal = SpSession.createDataFrame(df_pd_canal)
df_pd_travel = pd.read_csv("https://datos.canarias.es/api/estadisticas/statistical-resources/v1.0/datasets/ISTAC/E16028B_000011/1.48.csv")
df_sp_travel = SpSession.createDataFrame(df_pd_travel)


## Vista de todas la Tablas

In [205]:
df_sp_travel.show(5, truncate=False)

+--------------+----------------+---------------+-----------------+--------------------------+---------------------+-------------+---------------+---------------------------------+------------+---------+-------------------------------+-----------------+---------------------+-----------------------+
|TIME_PERIOD#es|TIME_PERIOD_CODE|TIPO_VIAJERO#es|TIPO_VIAJERO_CODE|LUGAR_RESIDENCIA#es       |LUGAR_RESIDENCIA_CODE|TERRITORIO#es|TERRITORIO_CODE|MEDIDAS#es                       |MEDIDAS_CODE|OBS_VALUE|CONFIDENCIALIDAD_OBSERVACION#es|NOTAS_OBSERVACION|ESTADO_OBSERVACION#es|ESTADO_OBSERVACION_CODE|
+--------------+----------------+---------------+-----------------+--------------------------+---------------------+-------------+---------------+---------------------------------+------------+---------+-------------------------------+-----------------+---------------------+-----------------------+
|01/2010       |2010-M01        |Turista        |TURISTA          |España (excluida Canarias)|ES_XIC

In [203]:
df_sp_known.show(5, truncate=False)
df_sp_money.show(5, truncate=False)

+---------------------------------------+--------------------+------------------------+--------+------------+-----------+----------------------+--------+-----------------+
|formas_conocer_Canarias                |paises_residencia_id|paises_residencia_nombre|islas_id|islas_nombre|periodos_id|periodos_nombre       |turistas|notas_observacion|
+---------------------------------------+--------------------+------------------------+--------+------------+-----------+----------------------+--------+-----------------+
|Conocía Canarias por visitas anteriores|T                   |Total países            |ES70    |Canarias    |2006Q1     |2006 Primer trimestre |1866124 |NaN              |
|Conocía Canarias por visitas anteriores|T                   |Total países            |ES70    |Canarias    |2006Q2     |2006 Segundo trimestre|1414361 |NaN              |
|Conocía Canarias por visitas anteriores|T                   |Total países            |ES70    |Canarias    |2006Q3     |2006 Tercer trimest

In [204]:
df_sp_stay.show(5, truncate=False)
df_sp_canal.show(5, truncate=False)

+--------------------+------------------------+------------------+--------+------------+-----------+----------------------+--------------+-----------------+
|paises_residencia_id|paises_residencia_nombre|tipos_alojamiento |islas_id|islas_nombre|periodos_id|periodos_nombre       |estancia_media|notas_observacion|
+--------------------+------------------------+------------------+--------+------------+-----------+----------------------+--------------+-----------------+
|T                   |Total países            |Total alojamientos|ES70    |Canarias    |2006Q1     |2006 Primer trimestre |10.28         |NaN              |
|T                   |Total países            |Total alojamientos|ES70    |Canarias    |2006Q2     |2006 Segundo trimestre|10.08         |NaN              |
|T                   |Total países            |Total alojamientos|ES70    |Canarias    |2006Q3     |2006 Tercer trimestre |10.47         |NaN              |
|T                   |Total países            |Total aloja

## Datos Travel


In [188]:
df_sp_travel.show(5, truncate=False)

+--------------+----------------+---------------+-----------------+--------------------------+---------------------+-------------+---------------+---------------------------------+------------+---------+-------------------------------+-----------------+---------------------+-----------------------+
|TIME_PERIOD#es|TIME_PERIOD_CODE|TIPO_VIAJERO#es|TIPO_VIAJERO_CODE|LUGAR_RESIDENCIA#es       |LUGAR_RESIDENCIA_CODE|TERRITORIO#es|TERRITORIO_CODE|MEDIDAS#es                       |MEDIDAS_CODE|OBS_VALUE|CONFIDENCIALIDAD_OBSERVACION#es|NOTAS_OBSERVACION|ESTADO_OBSERVACION#es|ESTADO_OBSERVACION_CODE|
+--------------+----------------+---------------+-----------------+--------------------------+---------------------+-------------+---------------+---------------------------------+------------+---------+-------------------------------+-----------------+---------------------+-----------------------+
|01/2010       |2010-M01        |Turista        |TURISTA          |España (excluida Canarias)|ES_XIC

In [113]:
df_sp_travel = df_sp_travel.withColumnRenamed("LUGAR_RESIDENCIA#es", "paises_residencia_nombre")

In [114]:
df_sp_travel = df_sp_travel.filter(
    (col("MEDIDAS#es") == "Turistas")
)


In [152]:
df_sp_travel.select("LUGAR_RESIDENCIA#es").distinct().show(20)

+--------------------+
| LUGAR_RESIDENCIA#es|
+--------------------+
|Comunidad Valenciana|
|           Andalucía|
|Resto de España (...|
|Castilla - La Mancha|
|            Cataluña|
|España (excluida ...|
|     Castilla y León|
|          País Vasco|
|             Galicia|
| Comunidad de Madrid|
+--------------------+



In [117]:
df_sp_travel = df_sp_travel.na.drop(subset=["OBS_VALUE"])

In [118]:
df_sp_travel.dtypes

[('TIME_PERIOD#es', 'string'),
 ('TIME_PERIOD_CODE', 'string'),
 ('TIPO_VIAJERO#es', 'string'),
 ('TIPO_VIAJERO_CODE', 'string'),
 ('paises_residencia_nombre', 'string'),
 ('LUGAR_RESIDENCIA_CODE', 'string'),
 ('TERRITORIO#es', 'string'),
 ('TERRITORIO_CODE', 'string'),
 ('MEDIDAS#es', 'string'),
 ('MEDIDAS_CODE', 'string'),
 ('OBS_VALUE', 'double'),
 ('CONFIDENCIALIDAD_OBSERVACION#es', 'double'),
 ('NOTAS_OBSERVACION', 'string'),
 ('ESTADO_OBSERVACION#es', 'string'),
 ('ESTADO_OBSERVACION_CODE', 'string')]

In [119]:
df_sp_travel.show(5, truncate=False)

+--------------+----------------+---------------+-----------------+--------------------------+---------------------+-------------+---------------+----------+------------+---------+-------------------------------+-----------------+---------------------+-----------------------+
|TIME_PERIOD#es|TIME_PERIOD_CODE|TIPO_VIAJERO#es|TIPO_VIAJERO_CODE|paises_residencia_nombre  |LUGAR_RESIDENCIA_CODE|TERRITORIO#es|TERRITORIO_CODE|MEDIDAS#es|MEDIDAS_CODE|OBS_VALUE|CONFIDENCIALIDAD_OBSERVACION#es|NOTAS_OBSERVACION|ESTADO_OBSERVACION#es|ESTADO_OBSERVACION_CODE|
+--------------+----------------+---------------+-----------------+--------------------------+---------------------+-------------+---------------+----------+------------+---------+-------------------------------+-----------------+---------------------+-----------------------+
|01/2010       |2010-M01        |Turista        |TURISTA          |España (excluida Canarias)|ES_XIC               |Lanzarote    |ES708          |Turistas  |TURISTAS    

In [120]:
df_sp_travel.select("MEDIDAS#es").distinct().show(truncate=False)

+----------+
|MEDIDAS#es|
+----------+
|Turistas  |
+----------+



## Datos Known


In [123]:
df_sp_known.describe().show()

+-------+-----------------------+--------------------+------------------------+--------+------------+------------------+--------------------+------------------+--------------------+
|summary|formas_conocer_Canarias|paises_residencia_id|paises_residencia_nombre|islas_id|islas_nombre|       periodos_id|     periodos_nombre|          turistas|   notas_observacion|
+-------+-----------------------+--------------------+------------------------+--------+------------+------------------+--------------------+------------------+--------------------+
|  count|                  14700|               14700|                   14700|   14700|       14700|             14700|               14700|             14700|               14700|
|   mean|                   NULL|                NULL|                    NULL|    NULL|        NULL|            2010.5|              2008.5|115202.76655733128|                 NaN|
| stddev|                   NULL|                NULL|                    NULL|    NULL|  

In [124]:
# df_spark.select("formas_conocer_Canarias").distinct().show(truncate=False)
df_sp_known.groupBy("formas_conocer_Canarias").count().show(truncate=False)

+--------------------------------------------+-----+
|formas_conocer_Canarias                     |count|
+--------------------------------------------+-----+
|Conocía Canarias por visitas anteriores     |2100 |
|Recomendación de amigos o familiares        |2100 |
|Folleto o catálogo del tour operador        |2100 |
|Recomendación de agencia de viajes          |2100 |
|La información obtenida a través de Internet|2100 |
|Otros                                       |2100 |
|No contesta                                 |2100 |
+--------------------------------------------+-----+



## Datos Money


In [202]:
df_sp_money.show(5, truncate=False)

+------------------------------------+--------------------+------------------------+------------------------------------------+----------------------------------------------+-----------+---------------------+-------------+-----------------+
|indicadores                         |paises_residencia_id|paises_residencia_nombre|principales_municipios_turisticos_islas_id|principales_municipios_turisticos_islas_nombre|periodos_id|periodos_nombre      |gastos_medios|notas_observacion|
+------------------------------------+--------------------+------------------------+------------------------------------------+----------------------------------------------+-----------+---------------------+-------------+-----------------+
|Gasto medio total por turista  y día|DE                  |Alemania                |ES70                                      |Canarias                                      |2006       |2006 ESTIMACIÓN ANUAL|96.84        |NaN              |
|Gasto medio total por turista  y dí

In [189]:
df_sp_money.columns

['indicadores',
 'paises_residencia_id',
 'paises_residencia_nombre',
 'principales_municipios_turisticos_islas_id',
 'principales_municipios_turisticos_islas_nombre',
 'periodos_id',
 'periodos_nombre',
 'gastos_medios',
 'notas_observacion']

In [190]:
df_sp_money = df_sp_money.withColumn(
    "gastos_medios",
    when(col("gastos_medios").rlike(r"^-?\d+(\.\d+)?$"), col("gastos_medios").cast("double"))
    .otherwise(None)
)

In [191]:
df_sp_money.dtypes

[('indicadores', 'string'),
 ('paises_residencia_id', 'string'),
 ('paises_residencia_nombre', 'string'),
 ('principales_municipios_turisticos_islas_id', 'string'),
 ('principales_municipios_turisticos_islas_nombre', 'string'),
 ('periodos_id', 'string'),
 ('periodos_nombre', 'string'),
 ('gastos_medios', 'double'),
 ('notas_observacion', 'string')]

In [192]:
df_sp_money.show()

+--------------------+--------------------+------------------------+------------------------------------------+----------------------------------------------+-----------+--------------------+-------------+-----------------+
|         indicadores|paises_residencia_id|paises_residencia_nombre|principales_municipios_turisticos_islas_id|principales_municipios_turisticos_islas_nombre|periodos_id|     periodos_nombre|gastos_medios|notas_observacion|
+--------------------+--------------------+------------------------+------------------------------------------+----------------------------------------------+-----------+--------------------+-------------+-----------------+
|Gasto medio total...|                   T|            Total países|                                      ES70|                                      Canarias|     2006Q1|2006 Primer trime...|       105.04|              NaN|
|Gasto medio total...|                   T|            Total países|                                    

In [194]:
df_sp_money.select("gastos_medios").distinct().show(5)

+-------------+
|gastos_medios|
+-------------+
|       106.01|
|       125.64|
|       120.34|
|        98.09|
|        95.57|
+-------------+
only showing top 5 rows


In [195]:
years_to_include = ["2006", "2007", "2008", "2009", "2010", "2011", "2012", "2013", "2014", "2015", "2016", "2017", "2018", "2019", "2020", "2021", "2022", "2023", "2024"]

df_sp_money = df_sp_money.filter(
    (col("periodos_id").isin(years_to_include)) 
    & (col("paises_residencia_nombre") != "Total países")

)


In [196]:
df_sp_money.show(5, truncate=False)

+------------------------------------+--------------------+------------------------+------------------------------------------+----------------------------------------------+-----------+---------------------+-------------+-----------------+
|indicadores                         |paises_residencia_id|paises_residencia_nombre|principales_municipios_turisticos_islas_id|principales_municipios_turisticos_islas_nombre|periodos_id|periodos_nombre      |gastos_medios|notas_observacion|
+------------------------------------+--------------------+------------------------+------------------------------------------+----------------------------------------------+-----------+---------------------+-------------+-----------------+
|Gasto medio total por turista  y día|DE                  |Alemania                |ES70                                      |Canarias                                      |2006       |2006 ESTIMACIÓN ANUAL|96.84        |NaN              |
|Gasto medio total por turista  y dí

In [197]:
df_sp_money.select("paises_residencia_nombre").distinct().show()

+------------------------+
|paises_residencia_nombre|
+------------------------+
|                  España|
|                Alemania|
|            Otros países|
|             Reino Unido|
+------------------------+



# ¿Cual es el gasto promedio por personas por día segun el pais de residencia del turista?

In [201]:
gasto_por_pais = df_sp_money.groupBy("paises_residencia_nombre") \
    .agg(
        avg("gastos_medios").alias("gasto_promedio"),
        sum("gastos_medios").alias("gasto_total"),
        count("*").alias("num_registros")
    ) \
    .orderBy(desc("gasto_promedio"))

print("Top 10 países por gasto promedio por persona por dia:")
gasto_por_pais.show(10, truncate=False)

Top 10 países por gasto promedio por persona por dia:
+------------------------+------------------+------------------+-------------+
|paises_residencia_nombre|gasto_promedio    |gasto_total       |num_registros|
+------------------------+------------------+------------------+-------------+
|Alemania                |415.45308103975555|543412.6300000002 |1320         |
|Otros países            |395.3835378787879 |521906.27         |1320         |
|Reino Unido             |331.25388888888887|435267.61         |1320         |
|España                  |304.0834166666667 |401390.11000000004|1320         |
+------------------------+------------------+------------------+-------------+



In [ ]:


#### Hay relacion entre el gasto y el tiempo de estadia? es decir, si se quedan mas tiempo gastan mas?
#### 